In [1]:
import os
import csv
import requests
import pytesseract
from PIL import Image
from io import BytesIO
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

d:\amzn ml challenge\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load constants
entity_unit_map = {
    'width': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'depth': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'height': {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard'},
    'item_weight': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'maximum_weight_recommendation': {'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton'},
    'voltage': {'kilovolt', 'millivolt', 'volt'},
    'wattage': {'kilowatt', 'watt'},
    'item_volume': {'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce', 'gallon', 
                    'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}
}

allowed_units = {unit for entity in entity_unit_map for unit in entity_unit_map[entity]}

In [4]:

# Function to download images from URLs
def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:
        return Image.open(BytesIO(response.content))
    return None

# Function to apply OCR and extract text
def extract_text_from_image(image):
    return pytesseract.image_to_string(image)

# Function to clean and extract value + unit
def extract_value_and_unit(ocr_text, entity_name):
    entity_units = entity_unit_map.get(entity_name, allowed_units)
    # Search for numbers + units in OCR text
    for unit in entity_units:
        if unit in ocr_text:
            # Extract the value before the unit
            try:
                value = float(ocr_text.split(unit)[0].strip())
                return f"{value} {unit}"
            except ValueError:
                continue
    return ""

# Hugging Face model pipeline for image captioning (optional)
def image_captioning(image):
    caption_pipeline = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
    caption = caption_pipeline(image)[0]['generated_text']
    return caption

In [6]:
# Initialize GPT-2 tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# GPT-2 doesn't have a pad_token by default, so we add one
tokenizer.pad_token = tokenizer.eos_token  # Use the eos_token as the pad_token

model = AutoModelForCausalLM.from_pretrained("gpt2")

d:\amzn ml challenge\venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
# Text generation function
def generate_entity_extraction(text, entity_name):
    # Prepare the input with attention mask
    inputs = tokenizer(
        f"Extract the {entity_name} from the following text:\n\n{text}\n\n",
        return_tensors="pt",  # Return as PyTorch tensors
        padding=True,         # Explicitly enable padding
        truncation=True,      # Truncate if the text is too long
        add_special_tokens=True  # Add special tokens if necessary
    )

    # Add attention mask to avoid warning
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Generate response
    output = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,  # Pass attention mask
        max_new_tokens=50,              # Set max tokens to control length
        do_sample=True,                 # Sampling for diverse outputs
        temperature=0.7                 # Adjust temperature for creativity
    )

    # Decode the generated tokens
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [8]:
# Main prediction function
def generate_predictions(test_file, output_file):
    # Read the test data
    with open(test_file, 'r') as test_f, open(output_file, 'w') as out_f:
        test_reader = csv.DictReader(test_f)
        fieldnames = ['index', 'prediction']
        writer = csv.DictWriter(out_f, fieldnames=fieldnames)
        writer.writeheader()

        for row in test_reader:
            index = row['index']
            image_url = row['image_link']
            entity_name = row['entity_name']
            
            # Download image
            image = download_image(image_url)
            if image is None:
                writer.writerow({'index': index, 'prediction': ''})
                continue

            # Apply OCR to extract text from image
            ocr_text = extract_text_from_image(image)

            # Optional: Generate image caption (if needed)
            caption = image_captioning(image)
            ocr_text += " " + caption  # Append caption to OCR results

            # Use OCR to extract value + unit
            prediction = extract_value_and_unit(ocr_text, entity_name)
            
            # If no prediction from OCR, attempt using local GPT-Neo model
            if not prediction:
                prediction = generate_entity_extraction(ocr_text, entity_name)

            writer.writerow({'index': index, 'prediction': prediction})


In [9]:
test_file_path = 'dataset/sample_test.csv'
output_file_path = 'sample_output.csv'
generate_predictions(test_file_path, output_file_path)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
d:\amzn ml challenge\venv\Lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
You may ignore this warning if your `pad_token_id` (50256) is identical to the `bos_token_id` (50256), `eos_token_id` (50256), or the `sep_token_id` (None), and your input is not padded.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to

In [11]:
import pandas as pd
import spacy


In [14]:
import spacy
from spacy.cli import download

# Attempt to load the spaCy model
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    # If loading fails, download and install the model
    download("en_core_web_sm")
    nlp = spacy.load("en_core_web_sm")

# Now you can use `nlp` as usual


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


d:\amzn ml challenge\venv\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [15]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define allowed units
allowed_units = {'centimetre', 'foot', 'inch', 'metre', 'millimetre', 'yard',
                  'gram', 'kilogram', 'microgram', 'milligram', 'ounce', 'pound', 'ton',
                  'kilovolt', 'millivolt', 'volt', 'kilowatt', 'watt',
                  'centilitre', 'cubic foot', 'cubic inch', 'cup', 'decilitre', 'fluid ounce',
                  'gallon', 'imperial gallon', 'litre', 'microlitre', 'millilitre', 'pint', 'quart'}

def extract_units(text, entity_type):
    # Process the text with spaCy
    doc = nlp(text)
    units = set()
    for token in doc:
        if token.text.lower() in allowed_units:
            units.add(token.text.lower())
    return units

def process_csv(input_file, output_file):
    # Read CSV
    df = pd.read_csv(input_file)
    
    results = []
    
    # Process each row
    for index, row in df.iterrows():
        prediction = row['prediction']
        if pd.isna(prediction) or not prediction:
            results.append({'index': index, 'prediction': ''})
            continue
        
        # Example entity types (can be expanded as needed)
        if 'width' in prediction.lower():
            entity_type = 'width'
        elif 'height' in prediction.lower():
            entity_type = 'height'
        elif 'depth' in prediction.lower():
            entity_type = 'depth'
        elif 'item_weight' in prediction.lower():
            entity_type = 'item_weight'
        elif 'voltage' in prediction.lower():
            entity_type = 'voltage'
        elif 'wattage' in prediction.lower():
            entity_type = 'wattage'
        elif 'item_volume' in prediction.lower():
            entity_type = 'item_volume'
        else:
            entity_type = None

        if entity_type:
            units = extract_units(prediction, entity_type)
            result = f"{entity_type}: " + ", ".join(units) if units else ""
            results.append({'index': index, 'prediction': result})
        else:
            results.append({'index': index, 'prediction': ''})

    # Save results to CSV
    result_df = pd.DataFrame(results)
    result_df.to_csv(output_file, index=False)

In [16]:
# Define input and output file paths
input_file = 'sample_output.csv'
output_file = 'processed_output.csv'

In [17]:
# Process the CSV file
process_csv(input_file, output_file)